<div style="background-color:#004d99; color:white; padding:30px; border-radius:10px; text-align:center;">
  <h1 style="margin:0; font-size:1.8em;">DATABASE DESIGN AND DEVELOPMENT CLASS ASSIGNMENT 3</h1>
</div>

<div  style="background-color:pink; padding:20px; text-align:left; border-radius:10px;">
<b>Design a University Course Registration System (10 marks). Use any OOP Language of your
choice, such as C++ or Python.</b></br>
<p></p>Objective:</br>
Implement an object-oriented data model to simulate a Course Registration System, where
students can register for courses, view their enrolled courses, and update personal or course
information.</p></br>
Assignment Requirements:</br>
<b><u>ENTITIES (CLASSES):</b></u></br>
<br>1.STUDENT</br>
    <ol>
        <li>Attributes:roll_no, department</li>
        <li>Methods:registerCourse(), viewCouerses(), updateProfile()</li>
    </ol>
<br>2.COURSE</br>
    <ol>
        <li>Attributes:course_code, course_name, credit_hours</li>
        <li>Methods:display()</li>
    </ol>
<br>1.ENROLLMENT</br>
    <ol>
        <li>Attributes:roll_no, course_code</li>
        <li>Acts as a bridge  class (like a junction table in DB)</li>
    </ol>
Tasks:</br>
<b><u>1. Design the Object Model:</b></u></br>
1. Define classes and their attributes/methods.</br>
2. Create relationships (e.g., one student → many courses).</br>
<b><u>2. Simulate the Database with Arrays:</b></u></br>
1. Allow students to input 3–5 students and 3–5 courses.</br>
2. Use arrays to store objects.</br>
<b><u>3. Implement Operations:</b></u></br>
1. Register a student for a course</br>
2. Display all courses a student is enrolled in</br>
3. Update student details (e.g., change name or department)</br>
4. Search for students or courses by ID</br>
</div>

### 🔹 Step 0: Set Up Database Connection

In [22]:
import mysql.connector

# Connect to the MySQL database
db = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Jack@11181',
    database="UniversitySystem"
)
cursor = db.cursor()

In [24]:
# ---------------------- STUDENT CLASS ----------------------
class Student:
    def __init__(self):
        self.roll_no = input("Enter student roll number: ")
        self.name = input("Enter student name: ")
        self.department = input("Enter student department: ")

    def register(self):
        cursor.execute(
            "INSERT INTO Student (roll_no, name, department) VALUES (%s, %s, %s)",
            (self.roll_no, self.name, self.department)
        )
        db.commit()
        print("✅ Student registered successfully!\n")

    @staticmethod
    def update_profile():
        roll_no = input("Enter roll number of student to update: ")
        new_name = input("Enter new name (or press Enter to skip): ")
        new_department = input("Enter new department (or press Enter to skip): ")

        if new_name:
            cursor.execute("UPDATE Student SET name = %s WHERE roll_no = %s", (new_name, roll_no))
        if new_department:
            cursor.execute("UPDATE Student SET department = %s WHERE roll_no = %s", (new_department, roll_no))
        
        db.commit()
        print("✅ Student profile updated.\n")

    @staticmethod
    def view_courses():
        roll_no = input("Enter student roll number to view courses: ")
        cursor.execute("""
            SELECT Course.course_code, Course.course_name 
            FROM Enrollment 
            JOIN Course ON Enrollment.course_code = Course.course_code 
            WHERE Enrollment.roll_no = %s
        """, (roll_no,))
        results = cursor.fetchall()

        if not results:
            print("❌ This student is not enrolled in any courses.")
        else:
            print("📚 Enrolled Courses:")
            for code, name in results:
                print(f"- {code}: {name}")


In [26]:
 #---------------------- COURSE CLASS ----------------------
class Course:
    def __init__(self):
        self.course_code = input("Enter course code: ")
        self.course_name = input("Enter course name: ")
        self.credit_hours = int(input("Enter credit hours: "))

    def register(self):
        cursor.execute(
            "INSERT INTO Course (course_code, course_name, credit_hours) VALUES (%s, %s, %s)",
            (self.course_code, self.course_name, self.credit_hours)
        )
        db.commit()
        print("✅ Course added successfully!\n")

In [28]:
# ---------------------- ENROLLMENT CLASS ----------------------
class Enrollment:
    def __init__(self):
        self.roll_no = input("Enter student roll number to enroll: ")
        self.course_code = input("Enter course code to register in: ")

    def enroll(self):
        # Prevent duplicate enrollment
        cursor.execute("SELECT * FROM Enrollment WHERE roll_no = %s AND course_code = %s",
                       (self.roll_no, self.course_code))
        if cursor.fetchone():
            print("⚠️ Student already enrolled in this course.")
        else:
            cursor.execute(
                "INSERT INTO Enrollment (roll_no, course_code) VALUES (%s, %s)",
                (self.roll_no, self.course_code)
            )
            db.commit()
            print("✅ Enrollment successful!\n")

In [30]:
# ---------------------- SEARCH FUNCTIONS ----------------------
def search_student():
    roll_no = input("Enter student roll number to search: ")
    cursor.execute("SELECT * FROM Student WHERE roll_no = %s", (roll_no,))
    student = cursor.fetchone()

    if student:
        print(f"🔍 Found: Roll No: {student[0]}, Name: {student[1]}, Department: {student[2]}")
    else:
        print("❌ Student not found.")

def search_course():
    course_code = input("Enter course code to search: ")
    cursor.execute("SELECT * FROM Course WHERE course_code = %s", (course_code,))
    course = cursor.fetchone()

    if course:
        print(f"🔍 Found: Course Code: {course[0]}, Name: {course[1]}, Credit Hours: {course[2]}")
    else:
        print("❌ Course not found.")

In [36]:
# ---------------------- MAIN MENU ----------------------
def main():
    while True:
        print("\n--- 🎓 University Course Registration System ---")
        print("1. Register a new student")
        print("2. Add a new course")
        print("3. Enroll a student in a course")
        print("4. View student’s enrolled courses")
        print("5. Update student profile")
        print("6. Search student by roll number")
        print("7. Search course by course code")
        print("8. Exit")

        choice = input("Choose an option (1-8): ")

        if choice == '1':
            student = Student()
            student.register()
        elif choice == '2':
            course = Course()
            course.register()
        elif choice == '3':
            enrollment = Enrollment()
            enrollment.enroll()
        elif choice == '4':
            Student.view_courses()
        elif choice == '5':
            Student.update_profile()
        elif choice == '6':
            search_student()
        elif choice == '7':
            search_course()
        elif choice == '8':
            print("👋 Goodbye!")
            break
        else:
            print("❌ Invalid choice. Please try again.")

# Run the program
main()

# Close connection after exiting
cursor.close()
db.close()


--- 🎓 University Course Registration System ---
1. Register a new student
2. Add a new course
3. Enroll a student in a course
4. View student’s enrolled courses
5. Update student profile
6. Search student by roll number
7. Search course by course code
8. Exit


Choose an option (1-8):  4
Enter student roll number to view courses:  S001


📚 Enrolled Courses:
- CS101: Introduction to Programming
- IT210: Networking Fundamentals

--- 🎓 University Course Registration System ---
1. Register a new student
2. Add a new course
3. Enroll a student in a course
4. View student’s enrolled courses
5. Update student profile
6. Search student by roll number
7. Search course by course code
8. Exit


Choose an option (1-8):  8


👋 Goodbye!
